In [3]:
from omegaconf import OmegaConf
config_path = "./logs/2023-11-05T03-32-03_custom_vqgan/configs/2023-11-05T03-32-03-project.yaml"
config = OmegaConf.load(config_path)
import yaml
print(yaml.dump(OmegaConf.to_container(config)))

data:
  params:
    batch_size: 2
    num_workers: 8
    train:
      params:
        size: 512
        training_images_list_file: /home/santoshsanjeev/vqgan/train.txt
      target: taming.data.custom.CustomTrain
    validation:
      params:
        size: 512
        test_images_list_file: /home/santoshsanjeev/vqgan/val.txt
      target: taming.data.custom.CustomTest
  target: main.DataModuleFromConfig
model:
  base_learning_rate: 4.5e-06
  params:
    ddconfig:
      attn_resolutions:
      - 24
      ch: 128
      ch_mult:
      - 1
      - 1
      - 2
      - 2
      - 4
      double_z: false
      dropout: 0.0
      in_channels: 3
      num_res_blocks: 2
      out_ch: 3
      resolution: 512
      z_channels: 256
    embed_dim: 256
    lossconfig:
      params:
        codebook_weight: 1.0
        disc_conditional: false
        disc_in_channels: 3
        disc_start: 30001
        disc_weight: 0.8
      target: taming.modules.losses.vqperceptual.VQLPIPSWithDiscriminator
    n_emb

In [5]:
from taming.models.vqgan import VQModel
model = VQModel(**config.model.params)

Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [6]:
import torch
ckpt_path = "/share/ssddata/santosh_models/data_from_G42/temporal_project/final_data/CVPR/mimic_vqgan/last.ckpt"
sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
missing, unexpected = model.load_state_dict(sd, strict=False)

In [7]:
model.cuda().eval()
torch.set_grad_enabled(False)

/home/santoshsanjeev/.conda/envs/taming/lib/python3.8/site-packages/torch/cuda/__init__.py:104: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [1]:
import torch
from PIL import Image
import numpy as np
import pickle
import albumentations

# Assuming your VQGanVAE class and required dependencies are defined in a module named vae
from vae import VQGanVAE

class YourTestClass:
    def __init__(self):
        self.indices_dict = {}  # Dictionary to store codebook indices
        self.saved_model_path = None  # Path to save the model

    # Function to load and preprocess the images
    def load_images_from_txt(self, file_path):
        # with open(file_path, 'r') as file:
        #     lines = file.readlines()
        return [file_path]

    # Function to process images, obtain codebook indices, and save the model
    def process_images_and_save_model(self, model_path, config_path, image_paths_file):
        # Load image file paths from a txt file
        file_paths = self.load_images_from_txt(image_paths_file)

        # Initialize the VQGanVAE model
        vqgan_model = VQGanVAE(vqgan_model_path=model_path, vqgan_config_path=config_path)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        vqgan_model.to(device)
        vqgan_model.eval()

        # Process each image and obtain codebook indices
        for i, file_path in enumerate(file_paths):
            try:
                image = Image.open(file_path)#.convert('RGB')
                if not image.mode == "RGB":
                    image = image.convert("RGB")
                image = np.array(image).astype(np.uint8)
                rescaler = albumentations.SmallestMaxSize(max_size = 512)
                cropper = albumentations.CenterCrop(height=512,width=512)
                preprocessor = albumentations.Compose([rescaler, cropper])

                image = preprocessor(image = image)
                image = (image['image']/127.5 - 1.0).astype(np.float32)

                img_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float().to(device)
                with torch.no_grad():
                    z_q, emb_loss, perplexity, indices = vqgan_model.get_codebook_indices(img_tensor)
                # Store indices in the dictionary
                # print(i, file_path, file_path.split('/')[-1][:-4])
                self.indices_dict[file_path.split('/')[-1][:-4]] = indices.cpu().numpy().tolist()[0]
                print(indices)
                print(f"Processed image {i + 1}/{len(file_paths)}")
                # decoded_image = vqgan_model.decode(indices)
                # decoded_image = decoded_image.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()
                # decoded_image = (decoded_image * 255).astype(np.uint8)
                # decoded_image_save_path = f"{'/l/users/santosh.sanjeev/logs/decoded_images'}/decoded_image_{i}.png"
                # Image.fromarray(decoded_image).save(decoded_image_save_path)
                # print(f"Decoded image saved: {decoded_image_save_path}")

            except Exception as e:
                print(f"Error processing image {i + 1}: {str(e)}")
        
        # with open('', 'wb') as f:
        #     pickle.dump(self.indices_dict, f)
        # Save the VQGanVAE model after processing images
        # torch.save(vqgan_model.state_dict(), save_model_path)
        # print(f"Model saved to {save_model_path}")
        # self.saved_model_path = save_model_path  # Store the saved model path for later use

    # Function to load the saved model and decode images
    def decode_and_save_images(self, codebook_indices_path, save_folder):
        if self.saved_model_path is None:
            print("No model path available. Please run 'process_images_and_save_model' first.")
            return
        
        # Load the saved model
        vqgan_model = VQGanVAE()
        vqgan_model.load_state_dict(torch.load(self.saved_model_path))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        vqgan_model.to(device)
        vqgan_model.eval()

        # # Load codebook indices from the pickle file
        # with open(codebook_indices_path, 'rb') as f:
        #     self.indices_dict = pickle.load(f)

        # Decode images and save them to the specified folder
        for image_key, indices in self.indices_dict.items():
            indices = torch.tensor(indices).to(device)
            decoded_image = self.decode(indices, vqgan_model)
            decoded_image = decoded_image.squeeze(0).permute(1, 2, 0).cpu().numpy()
            decoded_image = (decoded_image * 255).astype(np.uint8)

            image_save_path = f"{save_folder}/{image_key}.png"
            Image.fromarray(decoded_image).save(image_save_path)
            print(f"Decoded image saved: {image_save_path}")
# Example usage:
if __name__ == "__main__":
    test_obj = YourTestClass()
    
    # Replace these paths with your actual paths
    model_path = '/share/ssddata/santosh_models/data_from_G42/temporal_project/final_data/CVPR/mimic_vqgan/last.ckpt'
    config_path = '/share/ssddata/santosh_models/data_from_G42/temporal_project/final_data/CVPR/mimic_vqgan/2021-12-17T08-58-54-project.yaml'
    image_paths_file = '/share/ssddata/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p10/p10999737/s52341872/5aea5877-40b40fee-5bccd163-ca1bf0ce-a95c213d.jpg'
    codebook_indices_path = './codebook_indices.pickle'


    save_folder = './decoded_images'

    # Process images and save the model
    test_obj.process_images_and_save_model(model_path, config_path, image_paths_file)


ModuleNotFoundError: No module named 'torch'